In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/snli/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
# stream data from text files
def data_generator(f_path, word2idx):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      text1 = [word2idx.get(w, len(word2idx)) for w in text1.split()]
      text2 = [word2idx.get(w, len(word2idx)) for w in text2.split()]
      yield ((text1, text2), label2idx[label])


def dataset(is_training, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.int32, tf.int32), tf.int32)
  _pads = ((0, 0), -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
def masked_attention(x, align, mask, tile_len):
  pad = tf.fill(tf.shape(align), float('-inf'))
  mask = tf.tile(tf.expand_dims(mask, 1), [1, tile_len, 1])
  align = tf.where(tf.equal(mask, 0), pad, align)
  align = tf.nn.softmax(align)
  return tf.matmul(align, x)


def soft_align_attention(x1, x2, mask1, mask2):
  align12 = tf.matmul(x1, x2, transpose_b=True)
  align21 = tf.transpose(align12, [0,2,1])
  x1_ = masked_attention(x2, align12, mask2, tf.shape(x1)[1])
  x2_ = masked_attention(x1, align21, mask1, tf.shape(x2)[1])
  return x1_, x2_

In [0]:
class AttentivePooling(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.kernel = tf.keras.layers.Dense(units=1,
                                        activation=tf.tanh,
                                        use_bias=False)

  
  def call(self, inputs, training=False):
    x, masks = inputs
    # alignment
    align = tf.squeeze(self.kernel(self.dropout(x, training=training)), -1)
    # masking
    paddings = tf.fill(tf.shape(align), float('-inf'))
    align = tf.where(tf.equal(masks, 0), paddings, align)
    # probability
    align = tf.nn.softmax(align)
    align = tf.expand_dims(align, -1)
    # weighted sum
    return tf.squeeze(tf.matmul(x, align, transpose_a=True), -1)

In [0]:
class FFNBlock(tf.keras.Model):
  def __init__(self, params, name):
    super().__init__(name = name)
    self.dropout1 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.fc1 = tf.keras.layers.Dense(params['global_units'], tf.nn.elu)
    self.dropout2 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.fc2 = tf.keras.layers.Dense(params['global_units'], tf.nn.elu)
  
  def call(self, inputs, training=False):
    x = inputs
    x = self.fc1(x)
    x = self.dropout1(x, training=training)
    x = self.fc2(x)
    x = self.dropout2(x, training=training)
    return x

In [0]:
class DAM(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    self.embedding = tf.Variable(np.load(params['embedding_path']), name='pretrained_embedding')
    
    self.attend_transform = FFNBlock(params, name='attend')
    
    self.compare_transform = FFNBlock(params, name='compare')
    
    self.attentive_pooling = AttentivePooling(params)
    
    self.aggregate_transform = FFNBlock(params, name='aggregate')
    
    self.out_linear = tf.keras.layers.Dense(params['num_labels'], name='out_linear')
  
  
  def call(self, inputs, training=False):
    x1, x2 = inputs
    
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    
    mask1 = tf.sign(x1)
    mask2 = tf.sign(x2)
    
    x1 = tf.nn.embedding_lookup(self.embedding, x1)
    x2 = tf.nn.embedding_lookup(self.embedding, x2)
    
    x1_, x2_ = self._attend(x1=x1, x2=x2, mask1=mask1, mask2=mask2, training=training)
    
    x1 = self._compare(x1, x1_, training=training)
    x2 = self._compare(x2, x2_, training=training)
    
    x = self._aggregate(x1=x1, x2=x2, mask1=mask1, mask2=mask2, training=training)
    
    x = self.out_linear(x)
    
    return x
  
  
  def _attend(self, x1, x2, mask1, mask2, training):
    x1 = self.attend_transform(x1, training=training)
    x2 = self.attend_transform(x2, training=training)
    return soft_align_attention(x1, x2, mask1, mask2)
  
  
  def _compare(self, x, x_, training):
    aggregate_fn = lambda x, x_: tf.concat((x,
                                            x_,
                                           (x - x_),
                                           (x * x_),), -1)
    x = aggregate_fn(x, x_)
    x = self.compare_transform(x, training=training)
    return x
    
  
  def _aggregate(self, x1, x2, mask1, mask2, training):
    features = []
    features.append(tf.reduce_max(x1, axis=1))
    features.append(tf.reduce_max(x2, axis=1))
    features.append(self.attentive_pooling((x1, mask1), training=training))
    features.append(self.attentive_pooling((x2, mask2), training=training))
    x = tf.concat(features, axis=-1)
    x = self.aggregate_transform(x, training=training)
    return x

In [0]:
params = {
    'model_dir': '../model/esim',
    'log_path': '../log/esim.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'embedding_path': '../vocab/word.npy',
    'vocab_path': '../vocab/word.txt',
    'batch_size': 32,
    'buffer_size': 200000,
    'global_units': 300,
    'dropout_rate': 0.2,
    'lr': 4e-4,
    'num_labels': 3,
    'eval_steps': 550152//32//10,
    'num_patience': 7,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [0]:
params['word2idx'] = get_vocab(params['vocab_path'])
params['vocab_size'] = len(params['word2idx']) + 1

model = DAM(params)
model.build([[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for ((text1, text2), label) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((text1, text2), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
      loss = tf.reduce_mean(loss)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
    if global_step % params['eval_steps'] == 0:
      # EVALUATION
      m = tf.keras.metrics.Accuracy()

      for ((text1, text2), label) in dataset(is_training=False, params=params):
        logits = model((text1, text2), training=False)
        y_pred = tf.argmax(logits, axis=-1)
        m.update_state(y_true=label, y_pred=y_pred)

      acc = m.result().numpy()
      logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
      history_acc.append(acc)

      if acc > best_acc:
        best_acc = acc
        # you can save model here
      logger.info("Best Accuracy: {:.3f}".format(best_acc))

      if len(history_acc) > params['num_patience'] and is_descending(history_acc):
        logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
        break

W0416 05:17:04.976160 140193631795072 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('attend/dense/kernel:0', TensorShape([300, 300])),
 ('attend/dense/bias:0', TensorShape([300])),
 ('attend/dense_1/kernel:0', TensorShape([300, 300])),
 ('attend/dense_1/bias:0', TensorShape([300])),
 ('compare/dense_2/kernel:0', TensorShape([1200, 300])),
 ('compare/dense_2/bias:0', TensorShape([300])),
 ('compare/dense_3/kernel:0', TensorShape([300, 300])),
 ('compare/dense_3/bias:0', TensorShape([300])),
 ('attentive_pooling/dense_4/kernel:0', TensorShape([300, 1])),
 ('aggregate/dense_5/kernel:0', TensorShape([1200, 300])),
 ('aggregate/dense_5/bias:0', TensorShape([300])),
 ('aggregate/dense_6/kernel:0', TensorShape([300, 300])),
 ('aggregate/dense_6/bias:0', TensorShape([300])),
 ('out_linear/kernel:0', TensorShape([300, 3])),
 ('out_linear/bias:0', TensorShape([3])),
 ('pretrained_embedding:0', TensorShape([20884, 300]))]
Reading ../data/train.txt


I0416 05:17:27.425542 140193631795072 interactiveshell.py:2882] Step 0 | Loss: 1.3159 | Spent: 22.5 secs | LR: 0.000400
I0416 05:17:30.749292 140193631795072 interactiveshell.py:2882] Step 50 | Loss: 1.1219 | Spent: 3.3 secs | LR: 0.000400
I0416 05:17:34.058638 140193631795072 interactiveshell.py:2882] Step 100 | Loss: 0.9826 | Spent: 3.3 secs | LR: 0.000400
I0416 05:17:37.367680 140193631795072 interactiveshell.py:2882] Step 150 | Loss: 0.9631 | Spent: 3.3 secs | LR: 0.000399
I0416 05:17:40.669033 140193631795072 interactiveshell.py:2882] Step 200 | Loss: 1.1528 | Spent: 3.3 secs | LR: 0.000399
I0416 05:17:43.991778 140193631795072 interactiveshell.py:2882] Step 250 | Loss: 1.3205 | Spent: 3.3 secs | LR: 0.000399
I0416 05:17:47.278701 140193631795072 interactiveshell.py:2882] Step 300 | Loss: 1.0362 | Spent: 3.3 secs | LR: 0.000399
I0416 05:17:50.582987 140193631795072 interactiveshell.py:2882] Step 350 | Loss: 0.8648 | Spent: 3.3 secs | LR: 0.000399
I0416 05:17:53.871752 140193631795

Reading ../data/test.txt


I0416 05:19:32.911067 140193631795072 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.738
I0416 05:19:32.912086 140193631795072 interactiveshell.py:2882] Best Accuracy: 0.738
I0416 05:19:35.030549 140193631795072 interactiveshell.py:2882] Step 1750 | Loss: 0.7682 | Spent: 13.9 secs | LR: 0.000393
I0416 05:19:38.329691 140193631795072 interactiveshell.py:2882] Step 1800 | Loss: 0.7999 | Spent: 3.3 secs | LR: 0.000393
I0416 05:19:41.880823 140193631795072 interactiveshell.py:2882] Step 1850 | Loss: 0.5097 | Spent: 3.5 secs | LR: 0.000393
I0416 05:19:45.669396 140193631795072 interactiveshell.py:2882] Step 1900 | Loss: 0.9091 | Spent: 3.8 secs | LR: 0.000392
I0416 05:19:49.436394 140193631795072 interactiveshell.py:2882] Step 1950 | Loss: 0.6158 | Spent: 3.8 secs | LR: 0.000392
I0416 05:19:52.794051 140193631795072 interactiveshell.py:2882] Step 2000 | Loss: 0.9407 | Spent: 3.4 secs | LR: 0.000392
I0416 05:19:56.240848 140193631795072 interactiveshell.py:2882] Step 2050 | Loss: 

...

I0416 08:09:50.363343 140193631795072 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.848
I0416 08:09:50.364419 140193631795072 interactiveshell.py:2882] Best Accuracy: 0.853
I0416 08:09:51.176096 140193631795072 interactiveshell.py:2882] Step 139250 | Loss: 0.3974 | Spent: 14.0 secs | LR: 0.000099
I0416 08:09:54.491142 140193631795072 interactiveshell.py:2882] Step 139300 | Loss: 0.5169 | Spent: 3.3 secs | LR: 0.000099
I0416 08:09:58.274478 140193631795072 interactiveshell.py:2882] Step 139350 | Loss: 0.3912 | Spent: 3.8 secs | LR: 0.000099
I0416 08:10:02.074705 140193631795072 interactiveshell.py:2882] Step 139400 | Loss: 0.4396 | Spent: 3.8 secs | LR: 0.000099
I0416 08:10:05.708921 140193631795072 interactiveshell.py:2882] Step 139450 | Loss: 0.4289 | Spent: 3.6 secs | LR: 0.000098
I0416 08:10:09.046898 140193631795072 interactiveshell.py:2882] Step 139500 | Loss: 0.5373 | Spent: 3.3 secs | LR: 0.000098
I0416 08:10:12.389183 140193631795072 interactiveshell.py:2882] Step 1

Reading ../data/test.txt


I0416 08:12:00.132162 140193631795072 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.850
I0416 08:12:00.133208 140193631795072 interactiveshell.py:2882] Best Accuracy: 0.853
I0416 08:12:02.998768 140193631795072 interactiveshell.py:2882] Step 141000 | Loss: 0.2290 | Spent: 14.0 secs | LR: 0.000097
I0416 08:12:06.325297 140193631795072 interactiveshell.py:2882] Step 141050 | Loss: 0.3065 | Spent: 3.3 secs | LR: 0.000097
I0416 08:12:09.658456 140193631795072 interactiveshell.py:2882] Step 141100 | Loss: 0.3509 | Spent: 3.3 secs | LR: 0.000097
I0416 08:12:12.986647 140193631795072 interactiveshell.py:2882] Step 141150 | Loss: 0.3983 | Spent: 3.3 secs | LR: 0.000097
I0416 08:12:16.321492 140193631795072 interactiveshell.py:2882] Step 141200 | Loss: 0.4258 | Spent: 3.3 secs | LR: 0.000097
I0416 08:12:19.646850 140193631795072 interactiveshell.py:2882] Step 141250 | Loss: 0.2177 | Spent: 3.3 secs | LR: 0.000097
I0416 08:12:22.999230 140193631795072 interactiveshell.py:2882] Step 1

Reading ../data/test.txt


I0416 08:14:09.316809 140193631795072 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.850
I0416 08:14:09.317911 140193631795072 interactiveshell.py:2882] Best Accuracy: 0.853
I0416 08:14:10.949578 140193631795072 interactiveshell.py:2882] Step 142700 | Loss: 0.3838 | Spent: 15.9 secs | LR: 0.000095
I0416 08:14:15.675529 140193631795072 interactiveshell.py:2882] Step 142750 | Loss: 0.2724 | Spent: 4.7 secs | LR: 0.000095
I0416 08:14:20.887420 140193631795072 interactiveshell.py:2882] Step 142800 | Loss: 0.3619 | Spent: 5.2 secs | LR: 0.000095
I0416 08:14:25.958459 140193631795072 interactiveshell.py:2882] Step 142850 | Loss: 0.2062 | Spent: 5.1 secs | LR: 0.000095
I0416 08:14:30.764572 140193631795072 interactiveshell.py:2882] Step 142900 | Loss: 0.3955 | Spent: 4.8 secs | LR: 0.000095
I0416 08:14:35.758565 140193631795072 interactiveshell.py:2882] Step 142950 | Loss: 0.2455 | Spent: 5.0 secs | LR: 0.000095
I0416 08:14:39.440113 140193631795072 interactiveshell.py:2882] Step 1